<a href="https://colab.research.google.com/github/otwn/Geospatial-Analysis-Examples/blob/master/Proximity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess

# GeoPandas
try:
    import geopandas as gpd
except ImportError:
    print('geopandas package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    import geopandas as gpd


# Geemap
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Google Colab
try: 
  import google.colab
  import geemap.eefolium as emap
except:
  import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()    

geopandas package not installed. Installing ...
geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TnaRBKjdTEvnv0brho1GGJbbjCu78nGjG7iv0AHe0w4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3gHlmpjIFgqP6avxb5eRX-S4SrF3CfoKCOZNCBlrJ64WlOBml9Sx_q0

Successfully saved authorization token.


In [10]:
collisions = gpd.read_file("/content/drive/My Drive/Colab Notebooks/Geospatial/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")

m_1 = emap.Map(center=[40.7, -74], zoom=11)

emap.folium.plugins.HeatMap(data=collisions[["LATITUDE", "LONGITUDE"]], radius=10).add_to(m_1)
m_1

DriverError: ignored

In [7]:
hospitals = gpd.read_file("/content/drive/My Drive/Colab Notebooks/Geospatial/nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

DriverError: ignored

In [ ]:
m_2 = emap.Map(center=[40.7, -74], zoom=11) 

# Your code here: Visualize the hospital locations
for idx, row in hospitals.iterrows():
    Marker([row["latitude"], row["longitude"]]).add_to(m_2)

In [ ]:
print(hospitals.crs)
print(collisions.crs)

In [ ]:
ten_kilo_range = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000).unary_union
outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: ten_kilo_range.contains(x))]

In [ ]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

In [ ]:
def best_hospital(collision_location):
    # Your code here
    distance = hospitals.geometry.distance(collision_location)
    name = hospitals.iloc[distance.idxmin()]["name"]
    return name

# Test your function: this should suggest CALVARY HOSPITAL INC
print(best_hospital(outside_range.geometry.iloc[0]))

In [ ]:
outside_range.geometry.apply(best_hospital).name.value_counts()


In [ ]:
highest_demand = outside_range.geometry.apply(best_hospital).value_counts().idxmax()


In [ ]:
lat_1 = 40.6629
long_1 = -73.7614

# Your answer here: proposed location of hospital 2
lat_2 = 40.6697
long_2 = -73.8547


new_df = pd.DataFrame(
    {'Latitude': [lat_1, lat_2],
      'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263)
# get new percentage
new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
new_my_union = new_coverage.geometry.unary_union
new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))
# make the map
m = emap.folium.Map(location=[40.7, -74], zoom_start=11) 
emap.folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
emap.folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m)
for idx, row in new_gdf.iterrows():
    emap.folium.Marker([row['Latitude'], row['Longitude']]).add_to(m)
emap.folium.plugins.HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
emap.folium.LatLngPopup().add_to(m)